In [2]:
##------------------------------------------------------
## STEP6: Calculating per pa summary stats from extracted csvs [FIGURE 4B] 
##------------------------------------------------------
library("terra")
library("dplyr")
library("sf")
#library("s3")
library("sp")
library("foreach")
library("stringr")
library("aws.s3")

terra 1.7.29


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE



In [23]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
iso3 <- "BRA"
gediwk <- 24
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(paste(f.path,"vl_GEDI-PA_2024/matching_func_2024.R",sep=""))
gedi_paf <- list.files(paste(f.path,"WDPA_extract/pa_stats_ALL/",sep=""), pattern=".csv", full.names = TRUE)
length(gedi_paf)
gedi_paid <- basename(gedi_paf) %>% readr::parse_number() %>% unique()
length(gedi_paid)

[1] 1621

[1] 1621

In [4]:
#length(unique(readr::parse_number(basename(gedi_paf))))
#gedi_paf[which(duplicated(readr::parse_number(basename(gedi_paf))))]


In [3]:
pa_stats_summary_ALL <- matrix(data=NA, nrow=0, ncol=20)
colnames(pa_stats_summary_ALL) <- c("pa_id","count_0","count_1",
                                    "meanrh98_0","meanrh98_1","sdrh98_0","sdrh98_1",
                                    "meanagbd_0","meanagbd_1","sdagbd_0","sdagbd_1",
                                    "wwfbiom_0","wwfbiom_1","wwfecoreg_0","wwfecoreg_1",
                                    "REGION_0","REGION_1","PFT_0","PFT_1","iso3")
                                    
write.csv(pa_stats_summary_ALL, row.names=FALSE,
          file=paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))


In [11]:
pa_stats_summary_ALL <- read.csv(paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))
#pa_stats_summary_ALL

for(i in 1489:length(gedi_paf)){
#for(i in 1:length(gedi_paf)){

    pa_stats_summary_ALL <- read.csv(paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))
    
    this_paf <- gedi_paf[i]
    cat(paste("i = ", i, " : calculating summary stats for PA",this_paf,"\n"))
    pa_metrics <- read.csv(this_paf) #%>% unique()
    
    if (length(table(pa_metrics$status))<2) {
        cat(iso3, this_paf, "has 0 protected or treatment \n")
    } else if (table(pa_metrics$status)[1]!=0 && table(pa_metrics$status)[2]!=0) {
    #filter the datafrme by number of shots in each cell, which is equivalent to the number of occurence of each unique UID code
##    tt <- table(pa_metrics$UID)
##    qcellid <- table(pa_metrics$UID)[tt>5] %>% names()
##    pa_metrics_filtered <- pa_metrics %>% dplyr::filter(UID %in% qcellid)
    #calc summary stats for each country 
    #pa_stats_summary <- pa_metrics_filtered %>%
        pa_stats_summary <- pa_metrics %>%
        group_by(status) %>%
        dplyr::mutate(pa_id=as.character(pa_id)) %>%
        dplyr::summarise(pa_id=na.omit(unique(pa_id)),
                       count=length(rh98),
                       meanrh98=mean(rh98, na.rm=TRUE), sdrh98=sd(rh98, na.rm=TRUE),
                       meanagbd=mean(agbd, na.rm=TRUE), sdagbd=sd(agbd, na.rm=TRUE),
                       wwfbiom=getmode(wwfbiom), wwfecoreg=getmode(wwfecoreg),
                       REGION=getmode(region), PFT=getmode(pft)) %>%
        tidyr::pivot_wider(names_from=status, values_from= setdiff(names(.),c("pa_id", "status")))
    pa_stats_summary$iso3 <- iso3

    pa_stats_summary_ALL <- rbind(pa_stats_summary_ALL, pa_stats_summary)
    
    write.csv(pa_stats_summary_ALL, row.names=FALSE,
          file=paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep="")) 
  }
}
cat("Done summarizing pa-level stats for region", iso3, "\n")  


i =  1489  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555637330_gedi_wk24_TSMBF.csv 
i =  1490  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555637331_gedi_wk24_TSMBF+TSGSS+M.csv 
i =  1491  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682066_gedi_wk24_TSGSS.csv 
i =  1492  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682068_gedi_wk24_TSMBF.csv 
i =  1493  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682072_gedi_wk24_TSMBF.csv 
i =  1494  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682075_gedi_wk24_TSMBF.csv 
i =  1495  : calculating summary stats for PA /projects/

In [12]:
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
pa_stats_summary <- read.csv(paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))

In [15]:
length(pa_stats_summary$pa_id)

[1] 1618

In [16]:
length(unique(pa_stats_summary$pa_id))

[1] 1618

In [19]:
pa_ids <- basename(list.files(paste(f.path,"/WDPA_extract/pa_stats_ALL/",sep=""))) %>% readr::parse_number() %>% unique()
length(pa_ids)

[1] 1621

In [21]:
setdiff(pa_ids, pa_stats_summary$pa_id)

[1] 555576324 555636622 555682258

In [24]:
pa_stats_summary_ALL <- read.csv(paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))
#pa_stats_summary_ALL

#for(i in 1489:length(gedi_paf)){
#for(i in 1:length(gedi_paf)){
for(i in c(which(gedi_paid == 555576324),
           which(gedi_paid == 555636622),
           which(gedi_paid == 555682258))){
print(i)

    pa_stats_summary_ALL <- read.csv(paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep=""))
    
    this_paf <- gedi_paf[i]
    cat(paste("i = ", i, " : calculating summary stats for PA",this_paf,"\n"))
    pa_metrics <- read.csv(this_paf) #%>% unique()
    
    if (length(table(pa_metrics$status))<2) {
        cat(iso3, this_paf, "has 0 protected or treatment \n")
    } else if (table(pa_metrics$status)[1]!=0 && table(pa_metrics$status)[2]!=0) {
    #filter the datafrme by number of shots in each cell, which is equivalent to the number of occurence of each unique UID code
##    tt <- table(pa_metrics$UID)
##    qcellid <- table(pa_metrics$UID)[tt>5] %>% names()
##    pa_metrics_filtered <- pa_metrics %>% dplyr::filter(UID %in% qcellid)
    #calc summary stats for each country 
    #pa_stats_summary <- pa_metrics_filtered %>%
        pa_stats_summary <- pa_metrics %>%
        group_by(status) %>%
        dplyr::mutate(pa_id=as.character(pa_id)) %>%
        dplyr::summarise(pa_id=na.omit(unique(pa_id)),
                       count=length(rh98),
                       meanrh98=mean(rh98, na.rm=TRUE), sdrh98=sd(rh98, na.rm=TRUE),
                       meanagbd=mean(agbd, na.rm=TRUE), sdagbd=sd(agbd, na.rm=TRUE),
                       wwfbiom=getmode(wwfbiom), wwfecoreg=getmode(wwfecoreg),
                       REGION=getmode(region), PFT=getmode(pft)) %>%
        tidyr::pivot_wider(names_from=status, values_from= setdiff(names(.),c("pa_id", "status")))
    pa_stats_summary$iso3 <- iso3

    pa_stats_summary_ALL <- rbind(pa_stats_summary_ALL, pa_stats_summary)
    
    write.csv(pa_stats_summary_ALL, row.names=FALSE,
          file=paste(f.path,"WDPA_extract/pa_stats/",iso3,"_pa_stats_summary_wk",gediwk,"_2.csv", sep="")) 
  }
}
#cat("Done summarizing pa-level stats for region", iso3, "\n")  


[1] 1266
i =  1266  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555576324_gedi_wk24_TSMBF.csv 
BRA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555576324_gedi_wk24_TSMBF.csv has 0 protected or treatment 
[1] 1471
i =  1471  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555636622_gedi_wk24_TSMBF.csv 
BRA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555636622_gedi_wk24_TSMBF.csv has 0 protected or treatment 
[1] 1553
i =  1553  : calculating summary stats for PA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682258_gedi_wk24_.csv 
BRA /projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/pa_stats_ALL//BRA_pa_555682258_gedi_wk24_.csv has 0 protected or treatment 
